# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['stopwords','punkt','wordnet'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, precision_score, recall_score, confusion_matrix
from sklearn.base import TransformerMixin

In [3]:
# load data from database
engine = create_engine('sqlite:///desaster_response.db')
df = pd.read_sql("SELECT * FROM desaster_response", engine)
X = df['message'] 
y = df.drop(['id','message','original','genre'],axis=1)

In [4]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # text lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # tokenizing text
    words = word_tokenize(text)
    # Removing stop words
    words = [x for x in words if x not in nltk.corpus.stopwords.words('english')]
    # Lemmatize
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words] # for nouns
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words] # for verbs
    
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
             ('tdidf', TfidfTransformer()),
             ('multi', MultiOutputClassifier(estimator=RandomForestClassifier()))
            ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
pipeline.fit(X_train,y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
result=[]
precision = []
recall=[]
f1=[]
for i in range(y_pred.shape[1]):
    result.append(classification_report(np.array(y_test.iloc[:,i]),y_pred[:,i]))
    precision.append(precision_score(np.array(y_test.iloc[:,i]),y_pred[:,i],average='weighted'))
    recall.append(recall_score(np.array(y_test.iloc[:,i]),y_pred[:,i],average='weighted'))
    f1.append(recall_score(np.array(y_test.iloc[:,i]),y_pred[:,i],average='weighted'))
df_result=pd.DataFrame(data=[precision,recall,f1],index=['precision','recall','f1'])
df_result

In [ ]:
df_result['mean'] = df_result.loc[['precision','recall','f1']].mean(axis=1)

In [ ]:
df_result.head()

In [ ]:
accuracy_score = (y_test==y_pred).mean()
print("mean accuracy:",accuracy_score.mean())

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {'multi__estimator__min_samples_leaf': [1,2],
              'multi__estimator__n_estimators': [5,10]}

cv = GridSearchCV(pipeline,parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train,y_train)

In [ ]:
cv.best_params_

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
class DenseTransformer(TransformerMixin):
    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [28]:
pipeline_ver2 = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
             ('tdidf', TfidfTransformer()),
             ('to_dense', DenseTransformer()),
             ('nb', MultiOutputClassifier(estimator=GaussianNB()))
            ])

In [21]:
pipeline_ver2 = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
             ('tdidf', TfidfTransformer()),
             ('multi', MultiOutputClassifier(estimator=svm.SVC()))
            ])

In [ ]:
pipeline_ver2.fit(X_train, y_train)

In [ ]:
y_pred_v2 = pipeline.predict(X_test)

In [ ]:
result_v2=[]
precision_v2 = []
recall_v2=[]
f1=_v2[]
for i in range(y_pred.shape[1]):
    result_v2.append(classification_report(np.array(y_test.iloc[:,i]),y_pred_v2[:,i]))
    precision_v2.append(precision_score(np.array(y_test.iloc[:,i]),y_pred_v2[:,i],average='weighted'))
    recall_v2.append(recall_score(np.array(y_test.iloc[:,i]),y_pred_v2[:,i],average='weighted'))
    f1_v2.append(recall_score(np.array(y_test.iloc[:,i]),y_pred_v2[:,i],average='weighted'))
df_result_v2=pd.DataFrame(data=[precision_v2,recall_v2,f1_v2],index=['precision','recall','f1'])
df_result_v2

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.